<a href="https://colab.research.google.com/github/kjinb1212/Falldown-detection-KD/blob/main/teacher_voting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split

from glob import glob
import os
import sys
from PIL import Image

from efficientnet_pytorch import EfficientNet
import pandas as pd

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, input_size, train = True, padding = True, normalize = False,
                 bright_ness = 0.2, hue = 0.15, contrast = 0.15, random_Hflip = 0.3, rotate_deg = 20):
        orig_normal_path = glob(os.path.join(root_dir, 'normal') + '/*.jpg')
        orig_fall_path = glob(os.path.join(root_dir, 'falldown') + '/*.jpg')
        orig_back_path = glob(os.path.join(root_dir, 'background') + '/*.jpg')
        
        normal_paths = []
        fall_paths = []
        back_paths = []
        
        for path in orig_normal_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                normal_paths.append(path)
                
        for path in orig_fall_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                fall_paths.append(path)
                
        for path in orig_back_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                back_paths.append(path)
                
        self.total_paths = normal_paths + fall_paths + back_paths
        self.labels = [0] * len(normal_paths) + [1] * len(fall_paths) + [2] * len(back_paths)
        
        transform = []
        if train:
            #transform.append(torchvision.transforms.ColorJitter(brightness=bright_ness, hue=hue, contrast=contrast))
            transform.append(torchvision.transforms.RandomHorizontalFlip(p=random_Hflip))
            #transform.append(torchvision.transforms.RandomCrop(224))
            transform.append(torchvision.transforms.RandomRotation(degrees=rotate_deg))
        transform.append(torchvision.transforms.ToTensor())
        if normalize:
            transform.append(torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
        if padding:
            transform.append(lambda x: torchvision.transforms.Pad(((128 - x.shape[2]) // 2, (128 - x.shape[1]) // 2), fill=0,
                                                     padding_mode="constant")(x))
        transform.append(torchvision.transforms.Resize((input_size, input_size)))
        self.transform = torchvision.transforms.Compose(transform)
        
        
    def __len__(self):
        return len(self.total_paths)

    def __getitem__(self, index):
        img = Image.open(self.total_paths[index])
        img = self.transform(img)
        return img, self.labels[index]

In [ ]:
INPUT_SIZE = 128
PADDING = False
NORMALIZE = False
BATCHSIZE = 128
NUMEPOCH = 100


test_data = CustomDataset(
    root_dir='validation',
    input_size=INPUT_SIZE, train=False, padding=PADDING, normalize=NORMALIZE,
    bright_ness=0, hue=01.5, contrast=0.15, random_Hflip=0, rotate_deg=0)

In [ ]:
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCHSIZE, shuffle=False, num_workers=70, drop_last=True)

# voting classifier inference

In [ ]:
def get_teacher_output(model, loader):
    model.eval()
    output = []
    with torch.no_grad():
        for data, _ in loader:
            data = data.to(device)
            output.append(model(data))
    torch.cuda.empty_cache()
    return output

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


ensamble_list = [['b0', 'b1'], ['b0', 'b2'], ['b0', 'b3'], ['b0', 'b4'], ['b1', 'b2'], ['b1', 'b3'], ['b1', 'b4'], ['b2', 'b3'], ['b2', 'b4'], ['b3', 'b4'],
                ['b0', 'b1', 'b2'], ['b0', 'b1', 'b3'], ['b0', 'b1', 'b4'], ['b0', 'b2', 'b3'], ['b0', 'b2', 'b4'], ['b0', 'b3', 'b4'], ['b1', 'b2', 'b3'],
                ['b1', 'b2', 'b4'], ['b2', 'b3', 'b4'], ['b0', 'b1', 'b2', 'b3'], ['b0', 'b1', 'b2', 'b4'], ['b0', 'b1', 'b3', 'b4'], ['b0', 'b2', 'b3', 'b4'], 
                ['b1', 'b2', 'b3', 'b4'], ['b0', 'b1', 'b2', 'b3', 'b4']]

logs = []
for teachers in ensamble_list:
    teacher_output = []
    for teacher_name in teachers:
        model_name = 'efficientnet-' + teacher_name
        torch.cuda.empty_cache()

        teacher_model = EfficientNet.from_pretrained(model_name, num_classes=3).to(device)
        teacher_model.load_state_dict(torch.load('3_model_weights/'+ model_name + '.pth'))

        teacher_output.append(get_teacher_output(teacher_model, test_loader))

    criterion = nn.CrossEntropyLoss()
    n_teacher = len(teachers)
    test_losses = []
    correct = 0
    total = 0
    for i, (data, label) in enumerate(test_loader):
        label = label.to(device)
        output = teacher_output[0][i]
        for j in range(1, n_teacher):
            output += teacher_output[j][i]
        output /= n_teacher
        loss = criterion(output, label)
        test_losses.append(loss.item())
        _, predict = torch.max(output.data, 1)
        correct += (predict == label).sum().item()
        total += label.size(0)
    test_loss = np.average(test_losses)
    test_acc = 100 * correct / total

    save_name = 'voting'
    for teacher_name in teachers:
        save_name += '-' + teacher_name
    s = save_name + "\t loss: {}\t acc: {}%".format(test_loss, test_acc)
    logs.append(s)

In [ ]:
for log in logs:
    print(log)